Sources:
https://karpathy.ai/zero-to-hero.html
https://www.youtube.com/watch?v=kCc8FmEb1nY

# Import libraries

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import time
import re

# Parameters

In [2]:
SEED = 15

# Dataset parameters
train_ratio = 0.99
sample_size = -1

# Training parameters
batch_size = 256*6 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 128
n_head = 6
n_layer = 2
dropout = 0.2
num_epochs = 1

# Move the model to the appropriate device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
torch.manual_seed(SEED)
torch.cuda.device_count()

1

# Load files

In [4]:
from datasets import load_dataset

# Define the path to the dataset
dataset_name = "20231101.fr"

# Load the dataset
raw_dataset = load_dataset("wikimedia/wikipedia", dataset_name)

c:\Users\hzf04d\Anaconda3\envs\llm2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 17/17 [00:02<00:00,  8.26it/s]


# Data preparation

In [5]:
raw_dataset['train'].num_rows

2564646

In [6]:
%%time
# Create training and evaluation datasets
if sample_size < 0:
    train_sample = int(raw_dataset['train'].num_rows * train_ratio)
    test_sample = int(raw_dataset['train'].num_rows * (1- train_ratio))
else:
    train_sample = int(sample_size * train_ratio)
    test_sample = int(sample_size * (1- train_ratio))
    
ds_train_test = raw_dataset['train'].train_test_split(train_size=train_sample, test_size=test_sample, seed=SEED)

# Convert Dataset into str using list comprehension and join method
train_text = ''.join([item['text'] for item in ds_train_test['train']])
val_text = ''.join([item['text'] for item in ds_train_test['test']])

# Define allowed characters: basic French characters, accented characters, digits, basic punctuation
allowed_chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéàèùâêîôûëïüôçÉÀÈÙÂÊÎÔÛËÏÜÇ ' + \
                '0123456789,;.!?:\'"()-'

# Use regular expression to filter only allowed characters
pattern = re.compile(f'[^{re.escape(allowed_chars)}]')
train_text = pattern.sub('', train_text)
val_text = pattern.sub('', val_text)

# here are all the unique characters that occur in this text
chars = sorted(list(set(train_text + val_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

# create a mapping from characters to integers
str_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [str_to_int[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([int_to_str[i] for i in l]) # decoder: take a list of integers, output a string

# Store datasets into torch tensors
train_data = torch.tensor(encode(train_text), dtype=torch.long)
val_data = torch.tensor(encode(val_text), dtype=torch.long)

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

: 

# Define loss function

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

: 

# GPT

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out
    
    
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
    
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

    
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
#         if not isinstance(idx, torch.Tensor):
#             print(type(idx))
#             print(idx)
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


: 

## Single GPU

In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, encoded_text, block_size):
        """
        Args:
            encoded_text (Tensor): Encoded text data (as a long tensor).
            block_size (int): The size of each sequence block to generate.
        """
        self.data = encoded_text
        self.block_size = block_size

    def __len__(self):
        # Subtract block_size to prevent overflow (can't create a block beyond the data length)
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        """
        Generates a single data pair (x, y).
        """
        # Ensure that the index is within the valid range
        idx = min(idx, len(self.data) - self.block_size - 1)
        x = self.data[idx:idx+self.block_size]
        y = self.data[idx+1:idx+self.block_size+1]
        return x, y
    
def get_dataloaders(train_dataset, val_dataset, batch_size: int = 64):    
    # Instantiate the DataLoaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
#         pin_memory=True,
#         pin_memory_device=device,
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        drop_last=True,
#         pin_memory=True,
#         pin_memory_device=device,
    )
    return train_dataloader, val_dataloader

: 

In [ ]:
train_dataset = CustomTextDataset(train_data, block_size)
val_dataset = CustomTextDataset(val_data, block_size)
train_dl, eval_dl = get_dataloaders(train_dataset, val_dataset, batch_size)

: 

In [ ]:
def generate_text(model, device, max_new_tokens):
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(model.generate(context, max_new_tokens=max_new_tokens)[0].tolist()))
    
def check_device(model, batch):
    print("Model device:", next(model.parameters()).device)
    for i, item in enumerate(batch):
        print(f"Batch item {i} device:", item.device)

: 

In [ ]:
%%time
model = GPTLanguageModel()
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Build the DataLoaders
train_dataset = CustomTextDataset(train_data, block_size)
val_dataset = CustomTextDataset(val_data, block_size)
train_dataloader, val_dataloader = get_dataloaders(train_dataset, val_dataset, batch_size)

: 

In [ ]:
%%time

# And now we can train the model
for epoch in range(num_epochs):
    model.train()
    for step, batch in tqdm(enumerate(train_dataloader)):
        batch = [item.to(device) for item in batch]  # Manually move to device
#         check_device(model, batch)
        
        # every once in a while evaluate the loss on train and val sets
        if step % eval_interval == 0:
            losses = estimate_loss()
            print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            generate_text(model, device, max_new_tokens=100)
            print('-' * 50)
        
        # evaluate the loss
        logits, loss = model(batch[0], batch[1])
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

# generate from the model
generate_text(m, device, max_new_tokens=500)
#open('more.txt', 'w').write(decode(model.generate(context, max_new_tokens=10000)[0].tolist()))

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 